In [1]:
# ft articles data [heading, link, date]

import requests
from bs4 import BeautifulSoup

news_data = []
keyword = 'exxon'
            

for page in range(1, 2):
    url = f"https://www.ft.com/search?q={keyword}&page={page}&sort=date&isFirstView=true"
    result = requests.get(url)
    result_content = result.content
    soup = BeautifulSoup(result_content, "lxml")
    
    for article in soup.findAll("div", {"class": "o-teaser"}):
        heading = article.find("div", {"class": "o-teaser__heading"}).get_text(strip=True) #find(text=True), find text is outdated
        link = article.find("div", {"class": "o-teaser__heading"}).find('a', href=True)
        date = article.find('time', {"class": "o-teaser__timestamp-date"})

        if heading and link and date:
            news_data.append([heading, link['href'], date.text.strip()])

for item in news_data:
    print(f"Heading: {item[0]}, Link: {item[1]}, Date: {item[2]}")

print(f"Total articles collected: {len(news_data)}")

Heading: Wall Street reports best quarter for investment banking in 2 years, Link: /content/970d86e0-e728-49e4-b300-670685a479a8, Date: July 17, 2024
Heading: Oil industry eyes petrochemicals profits amid uncertain fossil fuel demandPremiumcontent, Link: /content/d1e582ea-4b08-418b-aa33-18dc943688c1, Date: July 8, 2024
Heading: Proxy season results show support for ESG efforts continues to ebb, Link: /content/1089ff59-47b9-41a2-83b1-98a205587b23, Date: July 5, 2024
Heading: Letter: Here’s my simple question about Guyana’s oil bonanza, Link: /content/c4f06b34-39f4-45d6-bda4-2c089cb44020, Date: July 1, 2024
Heading: Keep faith in China, Li Qiang tells business at ‘Summer Davos’, Link: /content/1f04f30c-a8c1-4b2e-83cc-dd0a4a8ee350, Date: June 28, 2024
Heading: Trump would be ‘devastating’ for offshore wind, warns Massachusetts governorPremiumcontent, Link: /content/0190f812-f6a0-4662-b8f8-a0c8077a70b0, Date: June 27, 2024
Heading: Business bridles at cost of green transition, Link: /conte

In [2]:
# set up langchain env 

import os
from dotenv import load_dotenv

load_dotenv()

LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [3]:
myurl = 'https://www.ft.com' + news_data[1][1]
print(myurl)

https://www.ft.com/content/d1e582ea-4b08-418b-aa33-18dc943688c1


In [4]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

myurl = 'https://www.ft.com' + news_data[0][1]

bs4_strainer = bs4.SoupStrainer(class_=("o-topper__headline o-topper__headline--large", "n-content-body js-article__content-body"))
loader = WebBaseLoader(
    web_paths=(myurl,),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

len(docs[0].page_content)

USER_AGENT environment variable not set, consider setting it to identify your requests.


5131

In [5]:
print(docs[0].page_content[:5000])

Wall Street has posted its best quarter for investment banking in more than two years, in what bankers said were the “early innings” of a sustained recovery.The five largest investment banks — Goldman Sachs, JPMorgan Chase, Morgan Stanley, Bank of America and Citigroup — together reported investment banking fees of $8.2bn in the second quarter, a 40 per cent increase from a year earlier and the highest since the start of 2022. All of the banks except Goldman announced higher than expected investment banking revenues for the quarter. Morgan Stanley chief financial officer Sharon Yeshaya said on Tuesday that as buyers and sellers started to close a valuation gap that has stymied deals, “we expect that we are still in the early innings of an investment banking rebound”, echoing a phrase used by Goldman chief executive David Solomon a day earlier.“The numbers look great,” said UBS analyst Brennan Hakwen, who said they “really fed into this narrative of capital markets reopening, which has 

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

9

In [17]:
len(all_splits[0].page_content)

1000

In [20]:
# all_splits[9].metadata

In [22]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [27]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("What was said about Exxon?")

len(retrieved_docs)
print(retrieved_docs[0].page_content)

as evidence of Gulf states’ long-term strategy.US oil majors have also been part of the action. Exxon is building its own petrochemical complex in southern China’s Guangdong province, as well as expanding its own chemical production at existing facilities on the US Gulf Coast.The drive for greener optionsAmid tough competition for the fastest-growing segment of global oil demand, some are betting that there is scope for further disruption — replacing petrochemicals with greener alternatives.At a refinery south-east of Paris, which was originally configured to take in crude oil, TotalEnergies is building a recycling facility. UK petrochemicals group Ineos, run by billionaire Sir Jim Ratcliffe, is developing a new ethylene cracker in the port of Antwerp that it claims would be the greenest in Europe, representing “the largest investment in the European chemical sector for a generation”.Meanwhile, in the US a crowd of start-ups have emerged that are seeking to convert biomass such as


In [28]:
from langchain_openai import ChatOpenAI
import getpass
import os
import langchainhub
from langchain import hub

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [29]:
prompt = hub.pull("rlm/rag-prompt")
print(prompt)

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages
print(example_messages[0].content)

input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("What was said about Exxon?"):
    print(chunk, end="", flush=True)

Exxon is building its own petrochemical complex in southern China’s Guangdong province and expanding its chemical production at existing facilities on the US Gulf Coast. The fight for market share in the petrochemicals business is heating up, with the explosion of demand for petrochemicals being overwhelmingly a Chinese story. US oil majors, including Exxon, are part of the action in the petrochemical sector.